In [ ]:
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM
from scipy.optimize import curve_fit
from natsort import natsorted
import matplotlib
font = {
        'weight' : 'bold',
        'size'   : 27}

matplotlib.rc('font', **font)
import h5py
import plotly.graph_objs as go
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
import pandas as pd


In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]
store_path = f"output/anderson/Plots_for_Report_20240613"

In [ ]:
a=5/3
slices = [0.01,0.03,0.05,0.07,0.09,0.1,0.15,0.3,0.5,0.7,0.9,1.0]


In [ ]:
def Td(L,T0,xi,l:float=1.12):
    Db = 1+2*3.25/xi
    Td = (1+3.25)*(l/Db)/xi *np.exp(-L/xi)
    return T0*Td

def Tb(L,l):
    return np.exp(-L/l) 

In [ ]:


for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"
      plt.figure(figsize=(20, 10))


      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue

            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                monitor_lambdas = a/(td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs))
                plt.plot(monitor_lambdas,np.log10(transmission_normalized), label=f"L={slices[i]}x18.01a")

      #Save the plot in PDF format
      store_path = f"output/anderson/Plots_for_Report_20240613"
      if not os.path.exists(store_path):
            os.makedirs(store_path)
            print(f"Folder '{store_path}' created successfully.")
      
      
      plt.xlabel(r'$\nu$')
      plt.ylabel('T')
      plt.legend()
      plt.grid()
      plt.title("Bandgap formation")
      #plt.savefig(f'{store_path}/Bandgap formation.pdf', format='pdf')
      plt.show()

      
    
        


     

               



In [ ]:
#Calculate T(L) for given frequencies 

frequencies_check = np.array([0.418,0.462,0.471])
transmission_total_l= []
for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                reduced_freqs = a/monitor_lambdas
                indices = [np.argmin(np.abs(reduced_freqs-item)) for item in frequencies_check]

                transmission_total_l +=  [transmission_normalized[indices]]

            


In [ ]:

frequencies_labels = np.array(["SAL","Near gap","PGB"])
l = [1.12,0.76,0.72]
xi = [8,3.6,5.1]
T0 = [1.,0.22,0.055]
z0 = 3.25


for j, nu in enumerate(frequencies_check):
    plt.figure(figsize=(20, 10))
    Ls = np.linspace(0.1,18.01,100)
    plt.plot(
             np.log10(np.array(slices)*18.01),
             np.log10(np.array(transmission_total_l)[:,j]), 
             "o")
    
    plt.plot(
             np.log10(Ls),
             np.log10(np.exp(-Ls)/(l[j])),
             '--',
             color="red")

    #popt, pcov = curve_fit(lambda L, T0,xi: Td(L,T0,xi,l=l[j]),(np.array(slices)*18.01)[3:], (np.array(transmission_total_l)[:,j])[3:])

    plt.plot(
        np.log10(Ls),
        np.log10(Td(Ls,T0[j],xi[j],l[j])),
                 '--',color="green")
    
    plt.title(f'$\\nu$={nu} - {frequencies_labels[j]}')
    plt.ylabel("T(L)")
    plt.xlabel("L/a")
   
    if not os.path.exists(store_path):
            os.makedirs(store_path)
            print(f"Folder '{store_path}' created successfully.")
      
      

    plt.grid()
    plt.savefig(f'{store_path}/T(L)_Monitor_Right_nu_{nu}-{frequencies_labels[j]}.pdf', format='pdf')
    plt.show()

In [39]:

frequencies_labels = np.array(["SAL","Near gap","PGB"])
l = [1.12,0.76,0.72]
xi = [8,3.6,5.1]
T0 = [1.,0.22,0.055]
z0 = 3.25


for j, nu in enumerate(frequencies_check):
    fig = go.Figure()
    Ls = np.linspace(0.1,18.01,100)
    fig.add_trace(go.Scatter(x=np.array(slices)*18.01, y=np.array(transmission_total_l)[:,j],
                                    mode='markers'
                                    ,showlegend =True,name="Simulations"
                            ))
    fig.add_trace(go.Scatter(x=Ls, y=np.exp(-Ls)/(l[j]),
                                    mode='lines'
                                    ,showlegend =False,name="Theory -Ballistic Component"
                            ))
    
    fig.add_trace(go.Scatter(x=Ls, y=Td(Ls,T0[j],xi[j],l[j]),
                                    mode='lines'
                                    ,showlegend =False,name="Theory - Localized Component"
                            ))
    
    
    
    
    
   
    if not os.path.exists(store_path):
            os.makedirs(store_path)
            print(f"Folder '{store_path}' created successfully.")
      
      

    fig.update_yaxes(type="log")
    fig.update_xaxes(type="log")
    fig.update_layout(
                title=dict(text=f'$a/\lambda=\\text{{{nu} - {frequencies_labels[j]}}}$'),
                xaxis_title=r"$\Large{L'/a}$",
                yaxis_title=r'$\large{\text{T(L)}}$',
                yaxis_range=[-1.5,0],
                font=dict(size=20),
                yaxis = dict(
                    showexponent = 'all',
                    exponentformat = 'e'
                    )
                
                )
    fig.write_image(f'{store_path}/T(L)_Monitor_Right_nu_{nu}-{frequencies_labels[j]}.png',format="png", engine="kaleido")
    fig.show()


<>:39: SyntaxWarning:

invalid escape sequence '\l'

<>:39: SyntaxWarning:

invalid escape sequence '\l'

C:\Users\HernandF\AppData\Local\Temp\ipykernel_8308\3204195624.py:39: SyntaxWarning:

invalid escape sequence '\l'



In [ ]:
for path_direction in [
                       "RCP Sample 1 Slices SAL Gap Formation/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                plt.figure(figsize=(15, 10))
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                transmission_flux_right = sim_data['flux1'].flux
                transmission_flux_left = sim_data['flux2'].flux
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                plt.plot(a/monitor_lambdas, np.log10(transmission_flux_left/transmission0), label='Left Monitor')
                plt.plot(a/monitor_lambdas, np.log10(transmission_flux_right/transmission0), label='Right Monitor')
                plt.xlabel(r'$\nu$')
                plt.ylabel('T')
                plt.legend()
                plt.grid()
                if not os.path.exists(store_path+'/convergence_tests'):
                  os.makedirs(store_path+'/convergence_tests')
                  print(f"Folder '{store_path+'/convergence_tests'}' created successfully.")
                plt.title(f"L={slices[i]}")
                plt.savefig(f'{store_path+'/convergence_tests'}/Convergence test L={slices[i]} 18.01a.pdf', format='pdf')
                plt.show()

               



In [ ]:
store_path = f"output/anderson/Plots_for_Report_20240613"

for path_direction in [
                       "RCP Sample 1 Long Run With Monitors/z_incidence"
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                plt.figure(figsize=(15, 10))
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                transmission_flux_right = sim_data['flux1'].flux
                transmission_flux_left = sim_data['flux2'].flux
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                plt.plot(1/monitor_lambdas, np.log10(transmission_flux_left/transmission0), label='Left Monitor')
                plt.plot(1/monitor_lambdas, np.log10(transmission_flux_right/transmission0), label='Right Monitor')
                plt.xlabel(r'$\nu$')
                plt.ylabel('T')
                plt.legend()
                plt.grid()
                if not os.path.exists(store_path):
                  os.makedirs(store_path)
                  print(f"Folder '{store_path}' created successfully.")
                plt.title(f"Convergence 18.01a Sample 1")
                plt.savefig(f'{store_path}/Convergence test L=18.01a.pdf', format='pdf')
                plt.show()

               



In [ ]:
print(structure_1.__str__())

In [ ]:
1/structure_1.monitor_lambdas

In [ ]:
sim_data.simulation.run_time*structure_1.fwidth

In [ ]:
store_path = f"output/anderson/Plots_for_Report_20240613"

for path_direction in [
                       "RCP Long Run Resolution 45/z_incidence"
                       ]:

      folder_path = f"data/{path_direction}"

      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                plt.figure(figsize=(15, 10))
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                print(structure_1.__str__())
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0 = sim_data0['flux1'].flux
                transmission = sim_data['flux1'].flux
                transmission_normalized = transmission / transmission0
                transmission_flux_right = sim_data['flux1'].flux
                transmission_flux_left = sim_data['flux2'].flux
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                print((1e-1/structure_1.monitor_lambdas[0],1e-1/structure_1.monitor_lambdas[-1]))
                plt.plot(1e-1/monitor_lambdas, np.log10(transmission_flux_left/transmission0), label='Left Monitor')
                plt.plot(1e-1/monitor_lambdas, np.log10(transmission_flux_right/transmission0), label='Right Monitor')
                plt.xlabel(r'$\nu$')
                plt.ylabel('T')
                plt.legend()
                plt.grid()
                if not os.path.exists(store_path):
                  os.makedirs(store_path)
                  print(f"Folder '{store_path}' created successfully.")
                plt.title(f"Convergence 18.01a Sample 1")
                plt.savefig(f'{store_path}/Convergence test {structure_1.run_time} runtime res {structure_1.resolution}  L=18.01a.pdf', format='pdf')
                plt.show()

               

